## **World Cup 2018 Prediction Test** 

Aim: Create model to see if predictions for World Cup 2018 results are accurate adn then to predict World Cup 2022 results in Summer 2022
Predicting the winners of the next FIFA World Cup

#### Filtering out irrelevant historic match data

The highest level of football before Senior football is U21 football. For national teams the AVG squad age is between 24 - 29.6 years. Players under the age of 21 rarely get recruited to the senior team so to accomodated for those that may 21 years old will be our lower bound. To accomodate for older players our upper bound will be 30 years. 

* It is rare for teams to stay together for longer than 5 years in any sport, some people get better in their game and stay others get worse and are dropped, some retire, some leave through injury. The list is endless, based on this number I will only collect data from up to 5 years before the day before the first game of the World Cup

* Because we want a clear view of the performance of the current squad and not the England team of 1966 for example because that isnt the team playing now. A limit of 9 years would help imrpove this bias.  
* Currently doing more research to validate any lower or higher number. I will simply update the function input when I get this infor. 
* Past performance of previous squads does not guarantee future performance 

In [92]:
# Importing the dependencies 
import pandas as pd
import numpy as np
from datetime import datetime
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### **----------------------------------  1. Data Collection  -----------------------------------**

In [94]:
# Reading data from JSON hyperlink to dataframe
df1 = pd.read_json("https://fixturedownload.com/feed/json/fifa-world-cup-2018")

# Exporting dataframe to csv
df1.to_csv('../Data/fixtures.csv', index=False)

,MatchNumber,RoundNumber,DateUtc,Location,HomeTeam,AwayTeam,Group,HomeTeamScore,AwayTeamScore
0,1,1,2018-06-14 15:00:00Z,"Luzhniki Stadium, Moscow",Russia,Saudi Arabia,Group A,5,0
1,2,1,2018-06-15 12:00:00Z,Ekaterinburg Stadium,Egypt,Uruguay,Group A,0,1
2,3,1,2018-06-15 15:00:00Z,Saint Petersburg Stadium,Morocco,Iran,Group B,0,1
3,4,1,2018-06-15 18:00:00Z,"Fisht Stadium, Sochi",Portugal,Spain,Group B,3,3
4,5,1,2018-06-16 10:00:00Z,Kazan Arena,France,Australia,Group C,2,1


In [93]:
# Loading the data from csv to dataframe
df = pd.read_csv('../Data/results.csv')

In [96]:
def recent_date() :
    """
    NO INPUT 
    """ 
    # LAST DAY OF DATA - Getting 5 years of data before World cup 2018
    recent_date = datetime.date(2018, 6, 13)

    # Converting the yr value to string to allow for calculation
    recent_date = str(recent_date)

    # Returning recent date
    return recent_date


def past_date(yr: int = 'test') :
    """
    yr -- Num of years before the first day of the World Cup 2018 
    """ 
    # LAST DAY OF DATA - Getting 5 years of data before World cup 2018
    recent_date = datetime.date(2018, 6, 13)

    # Converting the yr value to string to allow for calculation
    recent_date = str(recent_date)
    datem = datetime.datetime.strptime(recent_date, "%Y-%m-%d")

    # Subtracting yr amount of years from LAST DAY OF DATA
    past_year = int(datem.year) - yr
    past_date = datetime.date(past_year, 6, 13)

    # As string as we want the output without the datetime definition before it
    return str(past_date)


In [97]:
pd.options.mode.chained_assignment = None  # default='warn' # Must be set in first line 

# Conditions to look at dates that are within 9 years from today
data = df[(df.date <= recent_date()) & (df.date >= past_date(5))]
data.drop(['neutral'],inplace=True, axis=1)
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country
35351,2013-06-14,Ghana,Ivory Coast,1.0,1.0,Friendly,Obuasi,Ghana
35352,2013-06-14,Guatemala,Argentina,0.0,4.0,Friendly,Guatemala,Guatemala
35353,2013-06-14,Libya,Togo,2.0,0.0,FIFA World Cup qualification,Tripoli,Libya
35354,2013-06-14,Moldova,Kyrgyzstan,2.0,1.0,Friendly,Tiraspol,Moldova
35355,2013-06-15,Botswana,Central African Republic,3.0,2.0,FIFA World Cup qualification,Lobatse,Botswana


In [98]:
#Adding goal difference and establishing who is the winner 
winner = []
for i in range (len(data['home_team'])):
    if data ['home_score'].iloc[i] > data['away_score'].iloc[i]:
        winner.append(data['home_team'].iloc[i])
    elif data['home_score'].iloc[i] < data['away_score'].iloc[i]:
        winner.append(data['away_team'].iloc[i])
    else:
        winner.append('Draw')
data['winning_team'] = winner

# adding goal difference column
data['goal_difference'] = np.absolute(data['home_score'] - data['away_score'])

In [99]:
# Viewing shape of data
data.shape

(4666, 10)

In [100]:
data.reset_index(drop=True, inplace=True)

In [101]:
# Viewing first 5 rows of new data
data.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,winning_team,goal_difference
0,2013-06-14,Ghana,Ivory Coast,1.0,1.0,Friendly,Obuasi,Ghana,Draw,0.0
1,2013-06-14,Guatemala,Argentina,0.0,4.0,Friendly,Guatemala,Guatemala,Argentina,4.0
2,2013-06-14,Libya,Togo,2.0,0.0,FIFA World Cup qualification,Tripoli,Libya,Libya,2.0
3,2013-06-14,Moldova,Kyrgyzstan,2.0,1.0,Friendly,Tiraspol,Moldova,Moldova,1.0
4,2013-06-15,Botswana,Central African Republic,3.0,2.0,FIFA World Cup qualification,Lobatse,Botswana,Botswana,1.0


In [102]:
#dropping columns that wll not affect matchoutcomes
df_teams = data.drop(['date', 'home_score', 'away_score', 'tournament', 'city', 'country', 'goal_difference'], axis=1)
df_teams.head()

,home_team,away_team,winning_team
0,Ghana,Ivory Coast,Draw
1,Guatemala,Argentina,Argentina
2,Libya,Togo,Libya
3,Moldova,Kyrgyzstan,Moldova
4,Botswana,Central African Republic,Botswana


In [103]:
#Building the model
#the prediction label: The winning_team column will show "2" if the home team has won, "1" if it was a tie, and "0" if the away team has won.

df_teams = df_teams.reset_index(drop=True)
df_teams.loc[df_teams.winning_team == df_teams.home_team,'winning_team']=2
df_teams.loc[df_teams.winning_team == 'Draw', 'winning_team']=1
df_teams.loc[df_teams.winning_team == df_teams.away_team, 'winning_team']=0

df_teams.head()

,home_team,away_team,winning_team
0,Ghana,Ivory Coast,1
1,Guatemala,Argentina,0
2,Libya,Togo,2
3,Moldova,Kyrgyzstan,2
4,Botswana,Central African Republic,2


In [104]:
#convert home team and away team from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Separate X and y sets
X = final.drop(['winning_team'], axis=1)
y = final["winning_team"]
y = y.astype('int')

# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [105]:
final.head()

,winning_team,home_team_Abkhazia,home_team_Afghanistan,home_team_Albania,home_team_Alderney,home_team_Algeria,home_team_American Samoa,home_team_Andorra,home_team_Angola,home_team_Anguilla,...,away_team_Wales,away_team_Western Armenia,away_team_Western Isles,away_team_Yemen,away_team_Ynys Môn,away_team_Yorkshire,away_team_Zambia,away_team_Zanzibar,away_team_Zimbabwe,away_team_Åland Islands
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
score = logreg.score(X_train, y_train)
score2 = logreg.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.651
Test set accuracy:  0.507


In [107]:
#adding Fifa rankings
#the team which is positioned higher on the FIFA Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "home_teams" column
#since there are no "home" or "away" teams in World Cup games. 

# Loading new datasets
ranking = pd.read_csv('datasets/fifa_rankings.csv') 
fixtures = pd.read_csv('datasets/fixtures.csv')

# List for storing the group stage games
pred_set = []

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/fifa_rankings.csv'

In [ ]:
# Based on those teams that participated in the 2018 World cup 
# narrowing to team patcipating in the world cup
worldcup_teams = ['Australia', ' Iran', 'Japan', 'Korea Republic', 
            'Saudi Arabia', 'Egypt', 'Morocco', 'Nigeria', 
            'Senegal', 'Tunisia', 'Costa Rica', 'Mexico', 
            'Panama', 'Argentina', 'Brazil', 'Colombia', 
            'Peru', 'Uruguay', 'Belgium', 'Croatia', 
            'Denmark', 'England', 'France', 'Germany', 
            'Iceland', 'Poland', 'Portugal', 'Russia', 
            'Serbia', 'Spain', 'Sweden', 'Switzerland']
data_teams_home = data[data['home_team'].isin(worldcup_teams)]
data_teams_away = data[data['away_team'].isin(worldcup_teams)]
data_teams = pd.concat((data_teams_home, data_teams_away))
data_teams.drop_duplicates()
data_teams.count()

date               3277
home_team          3277
away_team          3277
home_score         3277
away_score         3277
tournament         3277
city               3277
country            3277
winning_team       3277
goal_difference    3277
dtype: int64

In [ ]:
# scapr teasm that played in WORLD CUP 2018 AND JUST KEEP TEAMS COL

# Based on participation last year, these are the only teasm that can 'win'


url = "https://fbref.com/en/comps/1/FIFA-World-Cup-Stats"
dfs = pd.read_html(url)

# For loop to locate desired table with len(df) > 5 | Renaming table to data 
for df in dfs:
    if len(df) > 15:
        table = df
        break
WCnations = table 

WCnations = WCnations['Squad']


WCnations.dropna(inplace=True)
WCnations= WCnations.str.split(" ", n = 1, expand = True) 
WCnations.rename({1 :'Nations'},axis=1, inplace=True)
WCnations.drop([0], axis=1, inplace=True)
WCnations.reset_index(drop=True, inplace=True)
WCnations

,Nations
0,France
1,Croatia
2,Belgium
3,England
4,Uruguay
5,Brazil
6,Sweden
7,Russia
8,Colombia
9,Denmark


In [ ]:
# Project will be completed 14th June when fixtures are released 